In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def gdf(data):
    return pd.read_csv(StringIO(data))

In [551]:
d="""datetime,A
2012-08-14 07:00,1
2012-08-14 07:30,300
2012-08-14 08:15,3
2013-08-14 16:00,600
2013-08-14 16:15,700"""
df1=gdf(d)
df1

,datetime,A
0,2012-08-14 07:00,1
1,2012-08-14 07:30,300
2,2012-08-14 08:15,3
3,2013-08-14 16:00,600
4,2013-08-14 16:15,700


In [552]:
d="""datetime,B
2012-08-14 07:00,100
2012-08-14 07:15,200
2012-08-14 07:30,300
2013-09-24 15:45, 50
2013-09-24 16:00, 60
2013-09-24 16:15, 70"""
df2=gdf(d)
df2

,datetime,B
0,2012-08-14 07:00,100
1,2012-08-14 07:15,200
2,2012-08-14 07:30,300
3,2013-09-24 15:45,50
4,2013-09-24 16:00,60
5,2013-09-24 16:15,70


In [553]:
df1.merge(df2, left_on=['datetime', 'A'], right_on=['datetime', 'B'],
         how='inner')

,datetime,A,B
0,2012-08-14 07:30,300,300


In [2]:
df = pd.DataFrame({
    'Date':[16234.0, 16003.]
})
#epoch = pd.to_datetime(0, unit='s').to_julian_date()
#pd.to_datetime(epoch - df.Date, unit='D')
pd.to_datetime('2016.01.03').to_julian_date()

pd.to_datetime(df.Date, format='%y%j')

0   2016-08-21
1   2016-01-03
Name: Date, dtype: datetime64[ns]

In [5]:
d = """col1,col2,col4
A,2,[2-3-4]
B,3,[2-6]
A,3,[2-3-4]
C,2,[2-3-4]
D,2,[2-3-4]"""
df=gdf(d)
df

,col1,col2,col4
0,A,2,[2-3-4]
1,B,3,[2-6]
2,A,3,[2-3-4]
3,C,2,[2-3-4]
4,D,2,[2-3-4]


In [6]:
df['col4'] = df.col4.astype(str).str.replace('-',',')
df['col2'] = df.col2.astype(str)
df= df[df.apply(lambda x: x.col2 in x.col4, axis=1)]
df

,col1,col2,col4
0,A,2,"[2,3,4]"
2,A,3,"[2,3,4]"
3,C,2,"[2,3,4]"
4,D,2,"[2,3,4]"


In [7]:
d="""Id,Valore
ID554,53.0
ID554,43.0
ID522,42.0
ID522,32.0
ID566,26.0"""
df=gdf(d)
df

,Id,Valore
0,ID554,53.0
1,ID554,43.0
2,ID522,42.0
3,ID522,32.0
4,ID566,26.0


In [21]:
df1=pd.DataFrame(
    np.random.randn(10),
    columns=['a'],
    index=pd.date_range('2020.01.01', periods=10)
)
df2=pd.DataFrame(
    np.random.randn(10),
    columns=['a'],
    index=pd.date_range('2020.01.04', periods=10)
)
df1.loc[['2020.01.01', '2020.01.04']]



,a
2020-01-01,0.026687
2020-01-04,0.018886


In [69]:
d="""c1,c2
-,75.0%
-5.5%,65.8%
-,6.9%"""
df=gdf(d)
df

,c1,c2
0,-,75.0%
1,-5.5%,65.8%
2,-,6.9%


In [71]:
df.replace(['%', '-','%'],['', 0.0, ''])
df.replace({'\%':'', '-$':0.0}, regex=True)
# Thanks to tdy
df.replace({'\%':'', r'^\s*-\s*$':0}, regex=True)
#df.replace()

,c1,c2
0,0,75.0
1,-5.5,65.8
2,0,6.9


In [44]:
d="""A
-0.5
-0.43
1
3.2
-0.97
6.3"""
df=gdf(d)
df

,A
0,-0.50
1,-0.43
2,1.00
3,3.20
4,-0.97
5,6.30


In [54]:
(df['A']>0)

0    False
1    False
2     True
3     True
4    False
5     True
Name: A, dtype: bool

In [52]:
df.A.mask(df.A<0,0)

0    0.0
1    0.0
2    1.0
3    3.2
4    0.0
5    6.3
Name: A, dtype: float64

In [62]:
df = pd.DataFrame({'a':[1,2,3,4,5,6,7,100,9],
                   'b':[1,2,3,4,5,6,7,8,9]})
df.rolling(3).corr()
df.corr(method='kendall')

,a,b
a,1.000000,0.944444
b,0.944444,1.000000


In [65]:
df['a'].rolling(3).corr(df.b)

0         NaN
1         NaN
2    1.000000
3    1.000000
4    1.000000
5    1.000000
6    1.000000
7    0.870619
8    0.018823
dtype: float64

In [106]:
datas = [['A', 51, 'id1', '2020-05-27 05:50:43.346'], ['A', 51, 'id2',
'2020-05-27 05:51:08.347'], ['B', 45, 'id3', '2020-05-24 17:23:55.142'],['B', 45, 'id4', '2020-05-24 17:23:30.141'], ['C', 34,
'id5', '2020-05-23 17:31:10.341']]

df = pd.DataFrame(datas, columns = ['col1', 'col2', 'cold_id',
'dates'])
df



,col1,col2,cold_id,dates
0,A,51,id1,2020-05-27 05:50:43.346
1,A,51,id2,2020-05-27 05:51:08.347
2,B,45,id3,2020-05-24 17:23:55.142
3,B,45,id4,2020-05-24 17:23:30.141
4,C,34,id5,2020-05-23 17:31:10.341


In [75]:
datas = [['A', 51, 'id1 id2', 'date_1'], ['B', 45, 'id3 id4',
'date_2'], ['C', 34, 'id5', 'date_3']]

dfo = pd.DataFrame(datas, columns = ['col1', 'col2', 'col_id',
'dates'])
dfo

,col1,col2,col_id,dates
0,A,51,id1 id2,date_1
1,B,45,id3 id4,date_2
2,C,34,id5,date_3


In [107]:
df['dates'] = pd.to_datetime(df.dates)
#df = df.set_index('dates')
df

,col1,col2,cold_id,dates
0,A,51,id1,2020-05-27 05:50:43.346
1,A,51,id2,2020-05-27 05:51:08.347
2,B,45,id3,2020-05-24 17:23:55.142
3,B,45,id4,2020-05-24 17:23:30.141
4,C,34,id5,2020-05-23 17:31:10.341


In [114]:
#df.groupby([df['dates'].dt.date]).agg(','.join)
df['dates'] = pd.to_datetime(df.dates)
df.groupby([pd.Grouper(key='dates', freq='2 min'), 
            'col1', 'col2']).agg(','.join).reset_index().sort_values('col1').reset_index(drop=True)


,dates,col1,col2,cold_id
0,2020-05-27 05:50:00,A,51,"id1,id2"
1,2020-05-24 17:22:00,B,45,"id3,id4"
2,2020-05-23 17:30:00,C,34,id5
